In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
import japanize_matplotlib


# 入出力ファイルの設定
INPUT_FILE_NAME = 'data.csv'
OUTPUT_IMAGE_NAME = 'lasso_coefficients.png'
OUTPUT_CSV_NAME = 'lasso_selected_variables.csv'

# フォント設定
plt.rcParams['font.size'] = 12

# カラム名の英語変換マッピング
rename_dict = {
    # 属性
    "年齢を教えてください。": "Age",
    "性別を教えてください。": "Gender",
    
    # 味覚項目
    "甘味をどの程度感じましたか？（１／５）": "Sweetness",
    "うま味をどの程度感じましたか？（２／５）": "Umami",
    "苦味をどの程度感じましたか？（３／５）": "Bitterness",
    "酸味をどの程度感じましたか？（４／５）": "Sourness",
    "塩味をどの程度感じましたか？（５／５）": "Saltiness",
    
    # 視覚項目
    "華やかさをどの程度感じましたか？（１／５）": "Floral",
    "開放感をどの程度感じましたか？（２／５）": "Openness",
    "爽やかさをどの程度感じましたか？（３／５）": "Freshness",
    "賑わい感をどの程度感じましたか？（４／５）": "Liveliness",
    "賑い感をどの程度感じましたか？（４／５）": "Liveliness",
    "高揚感をどの程度感じましたか？（５／５）": "Excitement",
    
    # 目的変数
    "クラフトビール「白蒲田」をまた飲んでみたいと思いますか？": "Drink_Again"
}

# 目的変数の指定
TARGET_VARIABLE = "Drink_Again"

# 分析から除外する変数
DROP_COLUMNS = [] 

# ==========================================
# メイン処理
# ==========================================

# データの読み込み
try:
    df = pd.read_csv(INPUT_FILE_NAME, encoding='shift_jis')
    print(f"データ読み込み成功: {INPUT_FILE_NAME}")
except FileNotFoundError:
    print(f"エラー: ファイル '{INPUT_FILE_NAME}' が見つかりません。")
    # ダミーデータ生成（動作確認用）
    df = pd.DataFrame(np.random.randint(1, 6, (50, 13)), columns=list(rename_dict.keys()))
    df['性別を教えてください。'] = np.random.choice(['男性', '女性'], 50)

# データ前処理
# カラム名の変更
df_renamed = df.rename(columns=rename_dict)

# 分析に必要なカラムのみを抽出
valid_cols = [col for col in rename_dict.values() if col in df_renamed.columns]
df_analysis = df_renamed[valid_cols]

# 数値型への変換（性別以外）
num_cols = [c for c in df_analysis.columns if c != 'Gender']
for col in num_cols:
    df_analysis[col] = pd.to_numeric(df_analysis[col], errors='coerce')

# 欠損値を含む行を削除
df_analysis = df_analysis.dropna()
print(f"分析に使用するデータ数（欠損除去後）: {len(df_analysis)}")

# 説明変数(X)と目的変数(y)の分割
X = df_analysis.drop(columns=[TARGET_VARIABLE])
y = df_analysis[TARGET_VARIABLE]

# カテゴリ変数（性別）のダミー変数化
# drop_first=True にすることで、「男性」フラグのみを作成（女性=0, 男性=1）し、多重共線性を防ぐ
X = pd.get_dummies(X, columns=['Gender'], drop_first=True)

print("\n説明変数（ダミー化後）:")
print(X.columns.tolist())

# データの標準化
# Lassoでは変数のスケールを合わせる必要がある
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
feature_names = X.columns # カラム名を保持

# Lasso回帰の実行 (LassoCV)
# 最適な alpha を自動探索
print("\nLasso回帰を実行中（最適パラメータ探索）...")
lasso = LassoCV(cv=5, random_state=42, n_alphas=100)
lasso.fit(X_scaled, y)

print(f"最適な alpha: {lasso.alpha_:.4f}")

# 結果の抽出
coef_df = pd.DataFrame({
    'Variable': feature_names,
    'Coefficient': lasso.coef_
})

# 係数の絶対値でソート
coef_df['Abs_Coefficient'] = coef_df['Coefficient'].abs()
coef_df = coef_df.sort_values(by='Abs_Coefficient', ascending=False)

# 選択された変数（係数が0でないもの）
selected_vars = coef_df[coef_df['Coefficient'] != 0]
removed_vars = coef_df[coef_df['Coefficient'] == 0]

print("\n【Lassoによって選択された変数】")
display(selected_vars[['Variable', 'Coefficient']].round(3))

print("\n【除外された変数（係数 = 0）】")
if len(removed_vars) > 0:
    display(removed_vars[['Variable', 'Coefficient']].round(3))
else:
    print("なし")

# 結果をCSV保存
selected_vars[['Variable', 'Coefficient']].・to_csv(OUTPUT_CSV_NAME, index=False)
print(f"選択された変数を保存しました: {OUTPUT_CSV_NAME}")

# 最適パラメータ選定プロセスの出力
print("\n--- 最適alphaの選定プロセス（上位5件） ---")
alphas = lasso.alphas_
mse_path = lasso.mse_path_
average_mse = np.mean(mse_path, axis=1)

cv_results = pd.DataFrame({
    'alpha': alphas,
    'Mean_MSE': average_mse
})
display(cv_results.sort_values('Mean_MSE').head(5))

# 係数の可視化
plt.figure(figsize=(10, 6))
colors = ['blue' if c > 0 else 'red' for c in coef_df['Coefficient']]
sns.barplot(x='Coefficient', y='Variable', data=coef_df, palette=colors)
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
plt.title(f"Lasso Coefficients (Target: {TARGET_VARIABLE})")
plt.xlabel("Coefficient Value (Standardized)")
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig(OUTPUT_IMAGE_NAME, dpi=300)
plt.show()